<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga de librerías</span>

In [2]:
import ultralytics
from ultralytics import YOLO
import os
import torch
import subprocess
import gdown
import json
import numpy as np
import subprocess

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de herramientas de etiquetado</span>

In [12]:
# Función para instalar los paquetes

def package_verification(package):
    try:
        output = os.popen('pip list').read()
        return f'{package}' in output
    except Exception as e:
        print(f'Error en la verificación: {e}')
        return False

def package_installation(package_name):
    if package_verification(package_name):
        print(f"El paquete '{package_name}' ya está instalado.")
    else:
        print('Instalando el paquete... ')
        try:
            os.system(f'pip install {package_name}')
            print(f'libreria {package_name} instalada correctamente! ')
        except Exception as e:
            print(f'Error en la instalación de {package_name}: {e}')

In [22]:
package_installation('labelme') # Instalación de la herramienta de etiquetado Labelme

El paquete 'labelme' ya está instalado.


In [21]:
# Abrir la herramienta para el proceso de etiquetado 
try:
    subprocess.check_call(["labelme"]) 
except Exception as e:
    print(f'No fue posible abrir labelme: {e}')

In [16]:
package_installation('labelme2yolo') # Instalación de la herramienta de conversión para el formato JSON de labelme a formato texto requerido por los modelos de detección de objetos YOLO.

El paquete 'labelme2yolo' ya está instalado.


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Conversión a formato YOLO</span>

In [ ]:
path_json= 'C:/Users/matrix/Workspace/lote2_2' # Carpeta que contiene las imagenes + archivo .json
dir_json_train='train' # Carpeta de entrenamiento, un 80% de imagenes.
dir_json_val='val' # Carpeta de validación, un 20% de imagenes.

try:
    subprocess.check_call(["labelme2yolo", "--json_dir", f'{path_json}/{dir_json_train}'])
    print(f'Conversión completada para la carpeta {path_json}/{dir_json_train}')
    subprocess.check_call(["labelme2yolo", "--json", f"{path_json}/{dir_json_val}"])
    print(f'Conversión completada para la carpeta {path_json}/{dir_json_val}')
except subprocess.CalledProcessError as e:
    print(f'Error al ejecutar función labelme2yolo: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de recursos</span>

In [19]:
package_installation('ultralytics') # Instalación de la biblioteca Ultralytics
#ultralytics.checks()

El paquete 'ultralytics' ya está instalado.


In [5]:
#Función para determinar si tiene GPU Cuda para instalación de PYTorch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'CUDA: {torch.cuda.get_device_name(0)}')
else:
    try:
        command = 'wmic path win32_videocontroller get caption'
        device = subprocess.check_output(command, shell=True, universal_newlines=True)
        print(f'Tarjeta Grafica: {device.strip()}.\n \nAdvertencia: Debe utilizar la CPU para instalación de PyTorch! ')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar el comando: {e}')

Tarjeta Grafica: Caption                  

AMD Radeon(TM) Graphics.
 
Advertencia: Debe utilizar la CPU para instalación de PyTorch! 


<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Dirección para instalar mediante el comando la versión de PyTorch en función de los requerimientos computacionales: </span>[Versión PyTorch](https://pytorch.org/get-started/locally/) 

In [ ]:
command_pytorch = 'pip3 install torch torchvision torchaudio' # Comando de instalación según la version personalizada a sus requerimientos computacionales
substrings = command_pytorch.split(" ")
try:
    subprocess.check_call(substrings)
except subprocess.CalledProcessError as e:
    print(f'Error al instalar la version de PyTorch: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación versión de modelo Yolo</span>

In [20]:
package_installation('gdown')

El paquete 'gdown' ya está instalado.


In [29]:
path_save= 'C:/Users/jemss/Workspace'
model_version= 'yolov8n.pt' # yolov8s.pt , yolov8m.pt , yolov8l.pt , yolov8x.pt
model_url= 'https://github.com/ultralytics/assets/releases/download/v0.0.0' 
def model_installation(url, version, save):
    try:
        model= f'{url}/{version}'
        gdown.download(model, save, quiet=False)
        print('Versión instalada correctamente! ')
    except Exception as e:
        print(f'Error en la instalación de la version del modelo: {e}')
    
model_installation(model_url, model_version, path_save)

Downloading...
From: https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt
To: C:\Users\jemss\Workspace
100%|██████████| 6.53M/6.53M [00:02<00:00, 2.73MB/s]

Versión instalada correctamente! 


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">ENTRENAMIENTO</span>

In [ ]:
model_version_YOLO='yolov8n.pt'

model_detection=YOLO(model_version_YOLO)

PROJECT='deteccion' # Le permite asignar un nombre al directorio de inicio que contendrá los experimentos de detección de objetos, y debe estar entre comillas; se recomienda no utilizar espacios en el nombre.
NAME='prueba_entrenamiento' # El nombre del experimento entrenamiento para deteccion de objetos debe ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
TASK='detect' # Define la tarea principal que desea realizar con el modelo YOLO v8; para este caso, la deteccion de objetos implica identificar la ubicación y la clase de objetos en una imagen o un flujo de video.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada. Puede especificarlo como un número entero, como imgsz a 640 para obtener un cuadrado perfecto, o como una tupla, como imgsz=(640,480) para establecer dimensiones específicas de ancho y alto. Se recomienda ajustar este valor según el tamaño del objeto que desea detectar. Para la detencción de objetos pequeños, se recomienda aumentar el valor a más de 640 píxeles para obtener una resolución más alta.
DATA="C:/Users/matrix/pruebayolo/data_copia_car/dataset.yaml" #  Le permite indicar la ruta al archivo que contiene los metadatos que utilizara el modelo detección de objetos y su configuración en formato YAML. Si especifica el valor data=None, el conjunto de datos coco128-seg.yaml se utiliza de forma predeterminada; de lo contrario, escriba la ruta al archivo YAML entre comillas utilizando barras diagonales (/) en lugar de barras invertidas (\).
EPOCHS=30 # Establezca el número de épocas del modelo YOLO v8 en la tarea de deteccion. Este valor representa el número total de iteraciones en todo el conjunto de datos de entrenamiento. Se recomienda experimentar con este parámetro dependiendo de la cantidad de imágenes disponibles. Si tiene un conjunto de datos grande, considere aumentar este valor por encima de 30 para obtener mejores resultados. Por otro lado, establecer epochs=None hará que el modelo continúe entrenándose hasta que la pérdida de validación deje de mejorar.
BATCH=-1 # Define la cantidad de imágenes procesadas simultáneamente en una iteración del modelo YOLO v8 en la deteccion. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
OPTIMIZER='auto' # Define el algoritmo de optimización para el modelo de deteccion YOLO. Su elección ajusta los pesos del modelo durante el entrenamiento y es crucial para la velocidad y rendimiento. Puede tomar valores como 'SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp' y 'auto', este último selecciona automáticamente el optimizador más adecuado a la tarea segmentación de objetos.
WORKERS=1 # Especifica el número de hilos de trabajo para la carga de datos en el modelo deteccion YOLO. Es recomendable utilizar un número de subprocesos que se ajusten al número de núcleos del CPU disponibles en el sistema.
DEVICE= 'cpu' # Especifica el dispositivo de ejecución para la versión del modelo yolov8 en la operación de deteccion. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante el entrenamiento de deteccion de objetos. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
SAVE=True # Cuando se establece en True, el modelo guarda puntos de control periódicamente durante el entrenamiento de deteccion. Se recomienda tener cuidado al establecer este valor en Verdadero, ya que está relacionado con el hiperparámetro SAVE_PERIOD; si establece el valor del hiperparametro a False, la función Save_Period se desactivará.
SAVE_PERIOD=-1 # Se utiliza para especificar con qué frecuencia se guardan los puntos de control durante el entrenamiento de deteccion. Si se establece en un valor mayor que 0, el modelo guardará puntos de control cada número especificado de épocas. Sin embargo, si save_period se establece en -1, significa que la función esta deshabilitada.
PATIENCE=30 # Representa el número esperado de épocas durante el entrenamiento de deteccion. Si no se observa mejora en el conjunto de validación dentro de un período específico, se detiene el proceso. Esta técnica de parada temprana se utiliza para evitar el sobreajuste del modelo. Se recomienda ajustar este hiperparámetro en función de la duración esperada del entrenamiento.
VERBOSE=True # Se utiliza para controlar el número de impresiones durante la ejecución del entrenamiento de deteccion. Para suprimir la salida de información básica únicamente, debe establecer el valor del hiperparámetro en False, pero si desea una salida de progreso más detallada, establezca el valor en True.
RECT= False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de entrenamiento en la deteccion de objetos. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente lote.
COS_LR=False # Reemplaza el decaimiento escalonado predeterminado de YOLOv8, que reduce la tasa de aprendizaje en ciertas épocas, con el decaimiento escalonado cos_lr. Este ajusta la tasa de aprendizaje según las épocas restantes y la tasa de aprendizaje inicial, proporcionando una disminución más suave. Establezca este hiperparámetro en True para una reducción gradual de la tasa de aprendizaje, de lo contrario establezca en False.
FRACTION= 1.0 # Controla la fracción del conjunto de datos que se utilizara para el entrenamiento de deteccion. Debes establecer este parámetro entre 0.0 y 1.0. Por defecto, cuando es 1.0, se emplea el 100% de las imágenes disponibles en el conjunto de datos.
EXIST_OK=False # Controla la sobreescritura de un experimento de deteccion existente. Cuando se establece en False, el sistema no sobrescribirá, en su lugar devolverá una ruta incrementada. Esto es útil para prevenir la sobreescritura accidental de experimentos anteriores. Para activar la función, asigna el valor True.

model_detection.train(project=PROJECT,name=NAME, task=TASK, data=DATA, imgsz=IMGSZ, epochs=EPOCHS, batch=BATCH, workers=WORKERS, device=DEVICE, plots=PLOTS, verbose=VERBOSE, rect=RECT, cos_lr=COS_LR, optimizer=OPTIMIZER, fraction=FRACTION, patience=PATIENCE, exist_ok=EXIST_OK)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">Configuración para la exportación</span>


A continuación se muestra una tabla de referencia para exportar un modelo YOLOv8 entrenado en la tarea deteccion de objetos. Tenga en cuenta que configurar el parámetro de `format` es fundamental para el proceso de exportación. Antes de continuar, asegúrese de verificar y ajustar estos valores a sus requisitos específicos: 
| Formatos                                                             | Asignación | Extensión                     | Hyperparámetros                                            | Descripción                                        |
|--------------------------------------------------------------------|-------------------|---------------------------|-----------------------------------------------------|----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | -                                                   | Modelo en formato PyTorch                           |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | "torchscript"     | `yolov8n.torchscript`     | `imgsz`, `optimize`                                 | Simplifica la implementación de modelos PyTorch en entornos de producción y aplicaciones eficientes, mejorando la portabilidad y el rendimiento al permitir ejecutar una representación intermedia en entornos sin Python.                       |
| [ONNX](https://onnx.ai/)                                           | "onnx"            | `yolov8n.onnx`            | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     | Desarrollado para promover la interoperabilidad, la optimización del hardware y la colaboración entre comunidades, al tiempo que responde a la necesidad de portabilidad de los modelos entre distintos marcos y herramientas de aprendizaje automático.                              |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | "openvino"        | `yolov8n_openvino_model/` | `imgsz`, `half`, `int8`                             | Elaborado para promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware Intel.                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | "engine"          | `yolov8n.engine`          | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` | Permite promover la interoperabilidad, la optimización del hardware y la implantación eficiente de modelos en distintos marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de NVIDIA.                     |
| [CoreML](https://github.com/apple/coremltools)                     | "coreml"          | `yolov8n.mlpackage`       | `imgsz`, `half`, `int8`, `nms`                      | Posibilita promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de Apple.                            |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | "saved_model"     | `yolov8n_saved_model/`    | `imgsz`, `keras`, `int8`                            | Empleado para guardar, compartir y desplegar modelos entrenados con TensorFlow. Versátil y facilita el despliegue en diversas plataformas como servidores, dispositivos móviles, embebidos y navegadores.                     |
| [TF Lite](https://www.tensorflow.org/lite)                         | "tflite"          | `yolov8n.tflite`          | `imgsz`, `half`, `int8`                             | Diseñado para el aprendizaje automático en dispositivos, TF Lite aborda restricciones clave como latencia, privacidad, conectividad, tamaño y consumo de energía. Es esencial para desplegar modelos en dispositivos móviles e integrados, ofreciendo una solución ligera y eficiente.                           |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | "edgetpu"         | `yolov8n_edgetpu.tflite`  | `imgsz`                                             | Utilizado para desplegar modelos de aprendizaje automático en el Edge TPU de TensorFlow. El Edge TPU es un pequeño ASIC (Circuito Integrado Específico de Aplicación) diseñado por Google para ofrecer inferencias de aprendizaje automático de alto rendimiento en dispositivos de bajo consumo.                       |
| [TF.js](https://www.tensorflow.org/js)                             | "tfjs"            | `yolov8n_web_model/`      | `imgsz`                                             | Facilita el despliegue de modelos de aprendizaje automático en navegadores web y Node.js, destacando la portabilidad y la facilidad de uso.                    |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | "paddle"          | `yolov8n_paddle_model/`   | `imgsz`                                             | Utilizado para desplegar modelos en PaddlePaddle, una plataforma de aprendizaje profundo de código abierto, paralela y distribuida que tiene su origen en la práctica industrial.                      |
| [ncnn](https://github.com/Tencent/ncnn)                            | "ncnn"            | `yolov8n_ncnn_model/`     | `imgsz`, `half`                                     | Formato optimizado para plataformas móviles, ofreciendo alto rendimiento. Puede incluir una estructura de archivo de modelo con información sobre capas, blobs de entrada y salida, y otros parámetros.

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">EXPORTACIÓN</span>

In [ ]:
trained_model_det="C:/Users/jemss/runs/detect/Pruebadeteccion/weights/best.pt"

selected_model_det=YOLO(trained_model_det)

FORMAT='onnx' # Seleccione el formato de exportación del modelo deteccion, empleando la tabla previamente proporcionada; en la columna "Asignación" se proporcionan las opciones para ajustar este valor.
INT8=True # Establezca este parámetro en True al utilizar la CPU y en False en caso contrario. La cuantificación a INT8 mejora la eficiencia del modelo deteccion en cuanto a memoria y velocidad de inferencia, especialmente en hardware que admite esta precisión.
HALF=False # Configúrelo en True cuando use la GPU; en caso contrario, False. La cuantificación a FP16 mejora la eficiencia de la memoria del modelo deteccion y la velocidad de inferencia, especialmente en hardware que admite precisión de punto flotante de 16 bits.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada para la exportación del modelo de deteccion. Puede especificarlo como un número entero, por ejemplo, 640 para un cuadrado perfecto, o como una tupla, por ejemplo, (640, 480) para dimensiones específicas de ancho y alto. Las imágenes que ingreses al modelo después de la exportación deben tener las mismas dimensiones específicas que has configurado para adaptarse a los requisitos del escenario de despliegue.
OPTIMIZE=False # Controla la optimización en modelos de detección a TorchScript para su implementación móvil. Es importante destacar que esta función puede resultar en un aumento significativo en el tamaño del modelo exportado, lo cual puede no ser ideal para aplicaciones móviles. Se configura con True para activar y False para desactivar.
DYNAMIC=False # Controla la habilitación de ejes dinámicos en modelos de deteccion, lo cual es particularmente útil para gestionar tamaños de lote variables. Esta característica funciona bien en escenarios donde el tamaño del lote puede cambiar durante la inferencia, como aplicaciones en tiempo real o de transmisión por secuencias. Los valores aceptados son Verdadero para habilitar la función y Falso para deshabilitarla.
SIMPLIFY=False # En la exportación de modelos de deteccion a ONNX|TensorRT, este hiperparámetro personaliza la complejidad del modelo controlando la optimización, eliminando capas redundantes y reduciendo la precisión de los parámetros. Se activa con True y se desactiva con False.
OPSET=False # Especifica la versión del conjunto de operadores en ONNX al exportar el modelo deteccion desde marcos como PyTorch o TensorFlow. Si se deja en "None", ONNX utilizará automáticamente la versión más reciente disponible; para una versión específica, asigne el número entre comillas, por ejemplo, "11".
WORKSPACE=4 # En la exportación de modelos de deteccion a TensorRT, establece el tamaño del espacio de trabajo en GB asignado para optimizar y preparar el modelo de red neuronal. Este espacio se utiliza durante la construcción del motor para lograr una ejecución eficiente en hardware GPU mediante la biblioteca TensorRT.
NMS=False # En la exportación de modelos de deteccion a CoreML, controla la inclusión de la Supresión No Máxima (NMS) en el modelo para eliminar cuadros delimitadores redundantes en la segmentación de instancias y mejorar la precisión de las predicciones. Establecer 'NMS' en 'False' ignora NMS en los modelos CoreML exportados.  Este ajuste, configurable durante la exportación del modelo YOLO, lo que permite a los usuarios optimizar la implementación del modelo en una variedad de plataformas y dispositivos.
KERAS= False # En la exportación de modelos de deteccion a TF SavedModel y TF Lite, permite optimizar el despliegue en diversas plataformas y dispositivos. Incluye también el formato del archivo, el dispositivo de ejecución y la posibilidad de manejar múltiples etiquetas por caja. Establezca el valor del hiperparámetro en True si está familiarizado con Keras; de lo contrario, en False para excluir su uso en la exportación.

selected_model_det.export(format=FORMAT, imgsz=640, dynamic=False, simplify=False, opset=False)


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">VALIDACIÓN</span>

In [ ]:
trained_model_det='C:/Users/matrix/pruebayolo/DETECCIÓN/Prueba_detección/weights/best.pt'

selected_model_det=YOLO(trained_model_det)

NAME='Prueba_validación_det' # El nombre del experimento de validación para deteccion de objetos deben ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
DATA= 'C:/Users/matrix/pruebayolo/data/dataset.yaml' #Permite indicar la ruta al archivo que contiene los metadatos necesarios para el proceso de validación, la ruta debe ser proporcionada entre comillas.
SAVE_JSON=True # Si se configura como True, habilita la funcionalidad de guardar los resultados obtenidos de manera detallada del proceso de validación en un formato estructurado JSON.
IMGSZ=640 # Establece las dimensiones en píxeles de la imagen de entrada para la validación del modelo de deteccion. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda usar el mismo valor que utilizo durante el entrenamiento del modelo.
BATCH=16 # Define la cantidad de imágenes procesadas simultáneamente en una iteración para la validación en detecciones de objetos. El valor predeterminado es 16; se recomienda establecerlo en -1 para utilizar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento y la eficiencia del entrenamiento, evitando problemas de memoria. Si desea personalizarlo, establezca el valor como un número entero.
SAVE_HYBRID=True # Activa la función con True para guardar una versión híbrida de la etiqueta, incluyendo la original y predicciones adicionales. Útil para el análisis detallado del rendimiento del modelo deteccion durante la validación; establezca en False para mostrar solo las predicciones.
CONF=0.5 # Establece el umbral de confianza para la validación de clases en la tarea de deteccion. Se recomienda un valor entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión. 
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara detectar en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
DEVICE='CPU' # Especifica el dispositivo de ejecución para la prueba de validación en la operación de deteccion. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante la validación en la deteccion de objetos. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
RECT=False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de validación en la deteccion de objetos. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente.
IOU=0.6 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,6. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren el mismo objeto. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.

selected_model_det.val(name=NAME, data=DATA, save_json=SAVE_JSON, imgsz=IMGSZ, batch=BATCH, save_hybrid=SAVE_HYBRID, conf=CONF, max_det=MAX_DET, device=DEVICE, plots=PLOTS, rect=RECT, iou=IOU, save_json=True)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de fuentes</span>

Para utilizar múltiples fuentes de datos al realizar predicciones con el modelo, se requiere que se ajuste el parámetro'source' a sus necesidades, tal como se indica en la siguiente tabla:
| Fuentes          | Asignación                             | Tipo             | Notas                                                           |
| --------------- | ------------------------------------ | ----------------- | --------------------------------------------------------------- |
| `image`           | 'image.jpg'                          | str or Path       | Archivo que contiene una única imagen.                                              |
| `URL`             | 'https://ultralytics.com/images/bus.jpg' | str               | Dirección que especifica la ubicación de una imagen en la web.                                                 |
| `screenshot`      | 'screen'                             | str               | El sistema captura la imagen actualmente visible en la pantalla y la utiliza como entrada para el modelo.                                           |
| `PIL`             | Image.open('im.jpg')                 | PIL.Image         | Utilizado para cargar imágenes en formato HWC (altura, ancho, canales) con canales RGB (rojo, verde y azul) mediante la biblioteca Python Imaging Library (PIL).                                   |
| `OpenCV`          | cv2.imread('im.jpg')                 | np.ndarray        | Permite la lectura de una imagen desde un archivo en formato HWC con canales BGR (azul, verde, rojo) utilizando la biblioteca OpenCV, almacenando la imagen como un array de NumPy.                    |
| `numpy`           | np.zeros((640,1280,3))               | np.ndarray        | Genera un array de ceros con las dimensiones especificadas para un formato HWC con canales BGR, utilizando la biblioteca NumPy.                    |
| `torch`           | torch.zeros(16,3,320,640)            | torch.Tensor      | Crea un tensor de ceros con las dimensiones especificadas para un formato HWC con canales RGB, empleando el framework PyTorch.               |
| `CSV`             | 'sources.csv'                        | str or Path       | Archivo de texto que almacena las rutas a las imágenes que se procesarán.   |
| `video`          | 'video.mp4'                          | str or Path       | Proporciona acceso a un archivo de video único.                       |
| `directory`      | 'path/'                              | str or Path       | Directorio que contiene múltiples archivos de imagen.               |
| `glob`           | 'path/*.jpg'                         | str               | Permite acceder a varias imágenes en un directorio usando expresiones de coincidencia de patrones. |
| `YouTube`        | 'https://youtu.be/LNwODJXcvt4'       | str               | Facilita el acceso a videos desde la plataforma YouTube.                                         |
| `stream`         | 'rtsp://example.com/media.mp4'      | str               | Permite la conexión a flujos de video o audio en tiempo real mediante protocolos como RTSP, RTMP, TCP o IP, ya sea a través de internet o una red local. |
| `multi-stream`   | 'list.streams'                       | str or Path       | Se utiliza para transmitir varios flujos de medios simultáneamente, permitiendo el procesamiento y análisis paralelo de múltiples flujos de medios. |

Formatos para las imágenes: 

| Image Suffixes | Reference                           |
| --------------- | ----------------------------------- |
| .bmp            | [Microsoft BMP File Format](https://docs.fileformat.com/es/image/bmp/)           |
| .dng            | [Adobe DNG](https://docs.fileformat.com/es/image/dng/)                           |
| .jpeg           | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .jpg            | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .mpo            | [Multi Picture Object](https://docs.fileformat.com/es/image/mpo/)                |
| .png            | [Portable Network Graphics](https://docs.fileformat.com/es/image/png/)           |
| .tif            | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .tiff           | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .webp           | [WebP](https://docs.fileformat.com/es/image/webp/)                                |
| .pfm            | [Portable FloatMap](https://docs.fileformat.com/font/pfm/)                   |


Formatos para los videos: 

| Video Suffixes | Reference                           |
| -------------- | ----------------------------------- |
| .asf           | [Advanced Systems Format](https://docs.fileformat.com/es/video/asf/)             |
| .avi           | [Audio Video Interleave](https://docs.fileformat.com/es/video/avi/)              |
| .gif           | [Graphics Interchange Format]()          |
| .m4v           | [MPEG-4 Part 14](https://docs.fileformat.com/es/video/m4v/)                      |
| .mkv           | [Matroska](https://docs.fileformat.com/es/video/mkv/)                            |
| .mov           | [QuickTime File Format](https://docs.fileformat.com/es/video/mov/)               |
| .mp4           | [MPEG-4](https://docs.fileformat.com/es/video/mp4/)          |
| .mpeg          | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .mpg           | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .ts            | [MPEG Transport Stream](https://docs.fileformat.com/es/video/ts/)               |
| .wmv           | [Windows Media Video](https://docs.fileformat.com/es/video/wmv/)                 |
| .webm          | [WebM Project](https://docs.fileformat.com/es/video/webm/)      

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">PREDICCIÓN</span>

In [ ]:
NAME='PruebaPrediccion_car' # El nombre del experimento de predicción para deteccion de objetos debe ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
SOURCE='C:/Users/matrix/pruebayolo/pruebaimagenes' # Establezca el origen de datos que el modelo de detección utilizará para realizar predicciones. Configure el valor de este hiperparámetro según la tabla proporcionada anteriormente.
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara predecir en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada durante la predicción en tareas de detección. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda utilizar los mismos valores utilizados durante el entrenamiento del modelo para mantener la coherencia en la inferencia.
CONF=0.5 # Establece el umbral de confianza durante el proceso de predicción en la tarea de detección. Se recomienda establecer el valor hiperparámetro entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión en la inferencia.
LINE_WIDTH= None # Determina el grosor en píxeles de los cuadros delimitadores que rodean los objetos detectados por el modelo. Puede establecer el grosor de la línea como un número entero en el que, a mayor valor, la línea será más gruesa, también puede utilizar como valor None para que el grosor se ajuste de forma automatizada, proporcionando una línea proporcional al tamaño de la imagen.
VISUALIZE=False # Determina si las características del modelo de detección deben mostrarse durante la predicción. Establecer esto en True permite que las características se muestren como mapas intermedias, lo que hace que el modelo sea más fácil de entender. Si se establece en False, no se mostrarán las características del modelo. 
IOU=0.7 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,7. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren la misma detección. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.
DEVICE='cpu' # Especifica el dispositivo de ejecución para la prueba de predicción en la operación de detección. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
VID_STRIDE=False # Controla la velocidad de los fotogramas durante el proceso de predicción en vídeos o secuencias de tiempo real. Al establecerlo en True el modelo se adapta a la velocidad de fotogramas especificada por la fuente de vídeo, procesando cada fotograma individualmente. Para desactivar esta función indique como valor False.
STREAM_BUFFER=False # Controla el almacenamiento en búfer de los fotogramas para la detección. Si es True, se almacenan todos los fotogramas para el procesamiento en tiempo real de vídeos o transmisiones en directo; si es False, devuelve el fotograma más reciente.
SAVE_FRAMES=False # Controla la captura y almacenamiento de los fotogramas predichos por el modelo de detección. Con True, se guardarán todos los fotogramas individuales predichos; con False, no se realizará el almacenamiento de los fotogramas.
AUGMENT=False # Aplica transformaciones a las imágenes de entrada, tales como giros, rotaciones, recortes y cambios de color, para diversificar los datos y mejorar la predicción en la detección. Establecer en True para activar la función, False para desactivar.
SAVE_CROP=True # Determina si se deben guardar imágenes recortadas con los resultados durante la predicción en la detección. Al establecerlo en "False", las imágenes recortadas no se guardarán, lo que reduce el tamaño del archivo. Con el valor "True", se guardarán las imágenes recortadas correspondientes a las áreas detectadas.
SHOW=False # Determina si se deben mostrar las imágenes o vídeos detectados durante la predicción. Al establecerlo en "True", permite la visualización de las predicciones en el mismo entorno, proporcionando una representación visual de los resultados. Si se establece en "False", las predicciones no se mostrarán. 

trained_model_det='C:/Users/matrix/pruebayolo/modelos/deteccion/experimentos/prueba_entrenamiento/weights/best.pt'
model_original= 'yolov8m.pt'

def train_classes(model_preentrenado, model_original):
    try:
        selected_model_det=YOLO(model_preentrenado)
        results_principal= selected_model_det.predict(name=NAME, source=SOURCE,conf=CONF, save_txt=True, max_det=MAX_DET, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, classes=None,  save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save=True, save_conf=True) 
        
        if results_principal is not None:
            
            selected_model_det_car=YOLO(model_original)
            results_car= selected_model_det_car.predict(name=NAME, source=SOURCE,conf=CONF, save_txt=True, max_det=MAX_DET, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, classes=2,  save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save=True, save_conf=True) 
        return results_principal, results_car
    
    except Exception as e:
        print(f'Error en el entrenamiento del modelo: {e}')

results_principal, results_car= train_classes(trained_model_det, model_original)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Función de área para cajas detectadas</span>

In [ ]:
# Función de área para formato pixel

def calcular_area(cajas_delimitadoras): 
    area_caja = [coordenada[2] * coordenada[3] for coordenada in cajas_delimitadoras] # Se asigna el ancho y alto a las coordenadas segun la posicion del formato x [0](coordenada x) y [1](coordenada y) w [2](ancho) h [3](alto) de las cajas delimitadoras. Para hallar el área de una caja delimitadora en la imagen, se recorren las coordenadas de la caja boxes y se multiplica la base por la altura.
    return sum(area_caja) # Si en la imagen se encuentra mas de una deteccion del mismo conjunto, realiza la suma de cada area individual.
     
def procesar_deteccion_individual(deteccion_base, deteccion_carro): 
    control_caja_carro = False # Se establece una variable de control para evitar obtener las coordenadas de la clase carro si en la imagen no hay detección de las clases principales. 
    boxes_rayon_abolladura = deteccion_base.boxes.xywh if deteccion_base.boxes is not None else None # Almacena las coordenadas de las cajas en formato xywh donde los valores estan en pixel, solo si la caja no esta vacia, de lo contrario retorna vacio. Para obtener las coordenadas en otro formato: xyxy(Representa las coordenadas xy de la esquina superior izquierda y la coordenada xy de la esquina inferior derecha); mismo formato pero los valor en pixel xyxywh. Otro formato es xywhn((Representa las coordenadas del centro de la caja, el ancho y alto de la caja) formato normalizado al tamaño de la imagen); en valores pixel xyxyn. 
    if len(boxes_rayon_abolladura) > 0: # Permite procesar las cajas solo si se obtiene el valor de las coordenadas.
        area_rayon_abolladura = calcular_area(boxes_rayon_abolladura) 
        print(f'Áreas rayón y abolladura: {area_rayon_abolladura}')
        
        for caja_carro in deteccion_carro: # Recorre los resultados que almacenan las cajas de deteccion para la clase carro.
            if deteccion_base.path == caja_carro.path: # Se verifica que la imagen en la que se halló la identificación de las cajas principales sea la misma en la que se esta recorriendo los resultados de deteccion carro.
                boxes_carro = caja_carro.boxes.xywh if caja_carro.boxes is not None else None # Al obtener las coordenas de la clase principal en un formato particular, se recomienda obtener las de la clase carro con el mismo formato, en este caso en pixel.
                control_caja_carro = True # Activa la variable de control solo si se obtuvieron las coordenadas de las cajas de deteccion carro.
                area_carro = calcular_area(boxes_carro) 
                print(f'Áreas carro: {area_carro}') 
        return (area_rayon_abolladura, area_carro) if control_caja_carro else (area_rayon_abolladura, 0) # Dependiendo del valor de la variable de control retornara los valores correspondientes a las dos areas, si no se obtuvo el valor de las coordenadas para carro en la misma imagen, solo se retornara el valor del area de la clase principal.      
    
    else:
        print(f'No se encontraron detecciones para la imagen! ') # Si la caja esta vacia no existira coordenadas para procesar por lo tanto retornara el valor 0 para el area afectada en cada area.
        return 0, 0
    
def procesar_detecciones(deteccion_central, deteccion_carro): # Recibe los resultados de la deteccion rayon-abolladura y las de carro.
    for id_objeto, caja_central in enumerate(deteccion_central, 0): # Permite recorrer las listas que contienen los resultados de las cajas principales, obteniendo el indice de posición y su valor.
        print(f'\nImagen {id_objeto + 1}: {caja_central.path} ') # Enumera las imagenes secuencialmente aumentando en 1, ademas proporciona la direccion de la imagen.
        area_rayon_abolladura, area_carro = procesar_deteccion_individual(caja_central, deteccion_carro) 
        area_total_objeto = abs(area_carro - area_rayon_abolladura) # Calcula el area total afectada del carro, proporcionando el resultado con el valor absoluto para evitar problemas cuando el area del carro sea igual a 0.
        print(f'área total afectada en la imagen: {area_total_objeto} ')

procesar_detecciones(results_principal,results_car)
print('\nProceso Terminado! ')